# Objective

1. Build a CNN model for Mnist dataset 
2. Adjust the parameters for better accuracy, such as number of layers, number of nodes in each layer, optimizer, learning rate, etc

# Prepare Environment

In [1]:
%env kERAS_BACKEND = tensorflow
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

env: kERAS_BACKEND=tensorflow


# Prepare Dataset

In [2]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Using TensorFlow backend.


In [3]:
import keras.utils as np_utils

x_train = x_train / 255
x_test = x_test / 25
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

# Build CNN

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam

def evaluate_param(feature_count=[32, 64, 128], conv_size=(3, 3), padding='same', activation='relu', pool_size=(2, 2), loss='mse', optimizer=SGD(lr=0.05)):
    model = Sequential()
    model.add(Conv2D(feature_count[0], conv_size, padding=padding, input_shape=(28, 28, 1)))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=pool_size))

    model.add(Conv2D(feature_count[1], conv_size, padding=padding))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=pool_size))

    model.add(Conv2D(feature_count[2], conv_size, padding=padding))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=pool_size))

    model.add(Flatten())
    model.add(Dense(200))
    model.add(Activation(activation))

    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    model.fit(x_train, y_train, batch_size=100, epochs=5)
    
    train_accuracy = model.evaluate(x_train, y_train)[1]
    test_accuracy = model.evaluate(x_test, y_test)[1]
    return (train_accuracy, test_accuracy)

# Tune Parameters
### Leave structure related parameters as default
    - Several options of each parameter are listed below, which lead to lots of permutations. It would take days to run all these permutations.
    - Moreover, exceptions often occur because the current layer does not have enough dimensions to process further dimension reduction.
    - To simply the problem, paramters affecting data dimensions or structures of the neural network are firstly left as default values, including feature_count, conv_size, padding and pool_size.

In [5]:
def run_tests(feature_counts, conv_sizes, paddings, activations, pool_sizes, losses, optimizers):
    accuracy_table = []
    for feature_count in feature_counts:
        for conv_size in conv_sizes:
            for padding in paddings:
                for activation in activations:
                    for pool_size in pool_sizes:
                        for loss in losses:
                            for optimizer in optimizers:
                                accuracy = -1.0
                                try:                                
                                    (train_accuracy, test_accuracy) = evaluate_param(feature_count, conv_size, padding, activation, pool_size, loss, optimizer)
                                except Exception as e:
                                    print('exception occurs!!', e)

                                accuracy_table.append({
                                    'train_accuracy': train_accuracy,
                                    'test_accuracy': test_accuracy,
                                    'feature_count': feature_count,
                                    'conv_size': conv_size,
                                    'padding': padding,
                                    'activation': activation,
                                    'pool_size': pool_size,
                                    'loss': loss,
                                    'optimizer': type(optimizer)
                                })

                                print('[', len(accuracy_table), ']', accuracy_table[-1])
    return accuracy_table

In [6]:
# structure related parameters
feature_counts = [
    #[16, 32, 64],
    #[32, 32, 32],
    [32, 64, 128],
    #[64, 64, 64],
    #[64, 128, 256],
    #[128, 128, 128],
]

conv_sizes = [
    #(2, 2),
    (3, 3),
    #(4, 4),
    #(5, 5),
    #(6, 6),
    #(7, 7),
    #(8, 8),
    #(9, 9),
    #(10, 10)
]

paddings = [
    #'valid',
    'same'
]

pool_sizes = [
    (2, 2),
    #(3, 3),
    #(4, 4),
    #(5, 5),
    #(6, 6),
    #(7, 7),
    #(8, 8),
    #(9, 9),
    #(10, 10)
]

1. Choose activation functions
    - Train models with fixed loss functions and optimizers.
    - To prevent from results biased by specific loss function or optimizer, 3 different loss functions and optimizers are choosen to create 9 tests for each activation function.
    - The choosen loose functions: mean_squared_error, hinge and categorical_crossentropy
    - The choosen optimizers: SGD, RMSprop and Adagrad, all with their default paramters

In [8]:
activations = [
    'softmax',
    'elu',
    'selu',
    'softplus',
    'softsign',
    'relu',
    'tanh',
    'sigmoid',
    'hard_sigmoid',
    'exponential',
    'linear'    
]

losses = [
    'mean_squared_error',
    'hinge',
    'categorical_crossentropy',
]

optimizers = [
    SGD(),
    RMSprop(),
    Adagrad(),
]

accuracy_table1 = run_tests(feature_counts, conv_sizes, paddings, activations, pool_sizes, losses, optimizers)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
activation_16 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
activation_17 (Activation)   (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 7, 7, 128)         73856     
__________

Epoch 1/5
60000/60000 [==============================] - 9s 154us/step - loss: 0.0900 - acc: 0.1118
Epoch 2/5
60000/60000 [==============================] - 9s 147us/step - loss: 0.0900 - acc: 0.1124
Epoch 3/5
60000/60000 [==============================] - 9s 148us/step - loss: 0.0900 - acc: 0.1124
Epoch 4/5
60000/60000 [==============================] - 9s 148us/step - loss: 0.0900 - acc: 0.1124
Epoch 5/5
10000/10000 [==============================] - 1s 76us/step
[ 3 ] {'train_accuracy': 0.11236666666666667, 'test_accuracy': 0.1135, 'feature_count': [32, 64, 128], 'conv_size': (3, 3), 'padding': 'same', 'activation': 'softmax', 'pool_size': (2, 2), 'loss': 'mean_squared_error', 'optimizer': <class 'keras.optimizers.Adagrad'>}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 28, 28, 32)        320       
________________________________________________________________

Epoch 1/5
60000/60000 [==============================] - 9s 158us/step - loss: 0.9899 - acc: 0.1122
Epoch 2/5
60000/60000 [==============================] - 9s 150us/step - loss: 0.9899 - acc: 0.1124
Epoch 3/5
60000/60000 [==============================] - 9s 149us/step - loss: 0.9898 - acc: 0.1124
Epoch 4/5
60000/60000 [==============================] - 9s 148us/step - loss: 0.9897 - acc: 0.1124
Epoch 5/5
10000/10000 [==============================] - 1s 76us/step
[ 6 ] {'train_accuracy': 0.11236666666666667, 'test_accuracy': 0.1135, 'feature_count': [32, 64, 128], 'conv_size': (3, 3), 'padding': 'same', 'activation': 'softmax', 'pool_size': (2, 2), 'loss': 'hinge', 'optimizer': <class 'keras.optimizers.Adagrad'>}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
activation_

Epoch 1/5
60000/60000 [==============================] - 10s 161us/step - loss: 2.3015 - acc: 0.1118
Epoch 2/5
60000/60000 [==============================] - 9s 150us/step - loss: 2.2796 - acc: 0.1368
Epoch 3/5
60000/60000 [==============================] - 9s 151us/step - loss: 1.9196 - acc: 0.3661
Epoch 4/5
60000/60000 [==============================] - 9s 151us/step - loss: 1.6699 - acc: 0.4018
Epoch 5/5
10000/10000 [==============================] - 1s 76us/step
[ 9 ] {'train_accuracy': 0.42065, 'test_accuracy': 0.4216, 'feature_count': [32, 64, 128], 'conv_size': (3, 3), 'padding': 'same', 'activation': 'softmax', 'pool_size': (2, 2), 'loss': 'categorical_crossentropy', 'optimizer': <class 'keras.optimizers.Adagrad'>}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
act

Epoch 1/5
60000/60000 [==============================] - 6s 94us/step - loss: 0.1798 - acc: 0.0993
Epoch 2/5
60000/60000 [==============================] - 5s 83us/step - loss: 0.1801 - acc: 0.0993
Epoch 3/5
60000/60000 [==============================] - 5s 82us/step - loss: 0.1801 - acc: 0.0993
Epoch 4/5
60000/60000 [==============================] - 5s 83us/step - loss: 0.1801 - acc: 0.0993
Epoch 5/5
10000/10000 [==============================] - 1s 62us/step
[ 12 ] {'train_accuracy': 0.0993, 'test_accuracy': 0.1032, 'feature_count': [32, 64, 128], 'conv_size': (3, 3), 'padding': 'same', 'activation': 'elu', 'pool_size': (2, 2), 'loss': 'mean_squared_error', 'optimizer': <class 'keras.optimizers.Adagrad'>}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_46 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
activation_76 (Act

...

Epoch 1/5
60000/60000 [==============================] - 9s 146us/step - loss: 0.9901 - acc: 0.0991
Epoch 2/5
60000/60000 [==============================] - 6s 99us/step - loss: 0.9901 - acc: 0.0991
Epoch 3/5
60000/60000 [==============================] - 6s 99us/step - loss: 0.9901 - acc: 0.0991
Epoch 4/5
60000/60000 [==============================] - 6s 99us/step - loss: 0.9901 - acc: 0.0991
Epoch 5/5
10000/10000 [==============================] - 1s 101us/step
[ 87 ] {'train_accuracy': 0.09913333333333334, 'test_accuracy': 0.098, 'feature_count': [32, 64, 128], 'conv_size': (3, 3), 'padding': 'same', 'activation': 'exponential', 'pool_size': (2, 2), 'loss': 'hinge', 'optimizer': <class 'keras.optimizers.Adagrad'>}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_271 (Conv2D)          (None, 28, 28, 32)        320       
_________________________________________________________________
activatio

Epoch 1/5
60000/60000 [==============================] - 9s 148us/step - loss: 1.1921e-07 - acc: 0.0987
Epoch 2/5
60000/60000 [==============================] - 6s 100us/step - loss: 1.1921e-07 - acc: 0.0987
Epoch 3/5
60000/60000 [==============================] - 6s 100us/step - loss: 1.1921e-07 - acc: 0.0987
Epoch 4/5
60000/60000 [==============================] - 6s 101us/step - loss: 1.1921e-07 - acc: 0.0987
Epoch 5/5
10000/10000 [==============================] - 1s 104us/step
[ 90 ] {'train_accuracy': 0.09871666666666666, 'test_accuracy': 0.098, 'feature_count': [32, 64, 128], 'conv_size': (3, 3), 'padding': 'same', 'activation': 'exponential', 'pool_size': (2, 2), 'loss': 'categorical_crossentropy', 'optimizer': <class 'keras.optimizers.Adagrad'>}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_280 (Conv2D)          (None, 28, 28, 32)        320       
_____________________________________

Epoch 1/5
60000/60000 [==============================] - 8s 139us/step - loss: 0.1799 - acc: 0.0984
Epoch 2/5
60000/60000 [==============================] - 5s 90us/step - loss: 0.1803 - acc: 0.0986
Epoch 3/5
60000/60000 [==============================] - 5s 91us/step - loss: 0.1803 - acc: 0.0986
Epoch 4/5
60000/60000 [==============================] - 5s 91us/step - loss: 0.1803 - acc: 0.0986
Epoch 5/5
10000/10000 [==============================] - 1s 98us/step
[ 93 ] {'train_accuracy': 0.09863333333333334, 'test_accuracy': 0.0958, 'feature_count': [32, 64, 128], 'conv_size': (3, 3), 'padding': 'same', 'activation': 'linear', 'pool_size': (2, 2), 'loss': 'mean_squared_error', 'optimizer': <class 'keras.optimizers.Adagrad'>}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_289 (Conv2D)          (None, 28, 28, 32)        320       
_________________________________________________________________
a

Epoch 1/5
60000/60000 [==============================] - 9s 143us/step - loss: 0.9898 - acc: 0.1022
Epoch 2/5
60000/60000 [==============================] - 6s 92us/step - loss: 0.9898 - acc: 0.1022
Epoch 3/5
60000/60000 [==============================] - 6s 93us/step - loss: 0.9898 - acc: 0.1022
Epoch 4/5
60000/60000 [==============================] - 6s 92us/step - loss: 0.9898 - acc: 0.1022
Epoch 5/5
10000/10000 [==============================] - 1s 104us/step
[ 96 ] {'train_accuracy': 0.10218333333333333, 'test_accuracy': 0.101, 'feature_count': [32, 64, 128], 'conv_size': (3, 3), 'padding': 'same', 'activation': 'linear', 'pool_size': (2, 2), 'loss': 'hinge', 'optimizer': <class 'keras.optimizers.Adagrad'>}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_298 (Conv2D)          (None, 28, 28, 32)        320       
_________________________________________________________________
activation_496

Epoch 1/5
60000/60000 [==============================] - 9s 149us/step - loss: 0.2612 - acc: 0.9341
Epoch 2/5
60000/60000 [==============================] - 6s 97us/step - loss: 0.0595 - acc: 0.9825
Epoch 3/5
60000/60000 [==============================] - 6s 96us/step - loss: 0.0417 - acc: 0.9876
Epoch 4/5
60000/60000 [==============================] - 6s 95us/step - loss: 0.0330 - acc: 0.9901
Epoch 5/5
10000/10000 [==============================] - 1s 108us/step
[ 99 ] {'train_accuracy': 0.9934833333333334, 'test_accuracy': 0.9483, 'feature_count': [32, 64, 128], 'conv_size': (3, 3), 'padding': 'same', 'activation': 'linear', 'pool_size': (2, 2), 'loss': 'categorical_crossentropy', 'optimizer': <class 'keras.optimizers.Adagrad'>}


2. Choose loss function
    - Train models with fixed activation functions and optimizers
    - To prevent from results biased by specific activation function or optimizer, 3 different activation functions and optimizers are choosen to create 9 tests for each loss function.
    - The choosen activation functions: softplus, relu, and hard_sigmoid
    - The choosen optimizers: Adadelta, Adam and Adamax, all with their default paramters

In [ ]:
activations = [
    'softplus',
    'relu',
    'hard_sigmoid',
]

losses = [
    'mean_squared_error',
    'mean_absolute_error',
    'mean_absolute_percentage_error',
    'squared_hinge',
    'hinge',
    'categorical_hinge',
    'logcosh',
    'categorical_crossentropy',
    'sparse_categorical_crossentropy',
    'binary_crossentropy',
    'kullback_leibler_divergence',
    'poisson',
    'cosine_proximity'
]

optimizers = [
    Adadelta(),
    Adam(),
    Adamax(),
]

accuracy_table2 = run_tests(feature_counts, conv_sizes, paddings, activations, pool_sizes, losses, optimizers)

3. Choose optimizer
    - Train models with fixed activation functions and loss functions
    - To prevent from results biased by specific activation function or loss function, 3 different activation functions and loss functions are choosen to create 9 tests for each optimizer.
    - The choosen activation functions: elu, tanh, and exponential
    - The choosen loss functions: squared_hinge, kullback_leibler_divergence and poisson

In [ ]:
activations = [
    'elu',
    'tanh',
    'exponential',
]

losses = [
    'squared_hinge',
    'kullback_leibler_divergence',
    'poisson',
]

optimizers = [
    SGD(),
    RMSprop(),
    Adagrad(),
    Adadelta(),
    Adam(),
    Adamax(),
    Nadam(),
]

accuracy_table3 = run_tests(feature_counts, conv_sizes, paddings, activations, pool_sizes, losses, optimizers)

4. Run with best paramters from the previous tests
    - According to the three tests above, even though not all of the permutations are tested, we can still find out some relatively robust and stable paramters to train our model.
    - The best and second best options for each parameter are listed in below
        - Activation functions: relu, softsign
        - Loss functions: categorical_crossentropy, poisson
        - Optimizers: RMSprop, Adamax
    - The best testing accuracy is 0.9928 with relu, categorical_crossentropy and Adamax.

In [10]:
activations = [
    'relu',
    'softsign',
]

losses = [
    'categorical_crossentropy',
    'poisson',
]

optimizers = [
    RMSprop(),
    Adamax(),
]

accuracy_table4 = run_tests(feature_counts, conv_sizes, paddings, activations, pool_sizes, losses, optimizers)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_307 (Conv2D)          (None, 28, 28, 32)        320       
_________________________________________________________________
activation_511 (Activation)  (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_307 (MaxPoolin (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_308 (Conv2D)          (None, 14, 14, 64)        18496     
_________________________________________________________________
activation_512 (Activation)  (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_308 (MaxPoolin (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_309 (Conv2D)          (None, 7, 7, 128)         73856     
__________

Epoch 1/5
60000/60000 [==============================] - 9s 158us/step - loss: 0.1176 - acc: 0.9433
Epoch 2/5
60000/60000 [==============================] - 6s 101us/step - loss: 0.1042 - acc: 0.9863
Epoch 3/5
60000/60000 [==============================] - 6s 102us/step - loss: 0.1029 - acc: 0.9910
Epoch 4/5
60000/60000 [==============================] - 6s 102us/step - loss: 0.1021 - acc: 0.9933
Epoch 5/5
10000/10000 [==============================] - 1s 112us/step
[ 3 ] {'train_accuracy': 0.9973, 'test_accuracy': 0.9904, 'feature_count': [32, 64, 128], 'conv_size': (3, 3), 'padding': 'same', 'activation': 'relu', 'pool_size': (2, 2), 'loss': 'poisson', 'optimizer': <class 'keras.optimizers.RMSprop'>}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_316 (Conv2D)          (None, 28, 28, 32)        320       
_________________________________________________________________
activation_526 (Activati

Epoch 1/5
60000/60000 [==============================] - 11s 179us/step - loss: 0.2056 - acc: 0.9400
Epoch 2/5
60000/60000 [==============================] - 7s 118us/step - loss: 0.0492 - acc: 0.9850
Epoch 3/5
60000/60000 [==============================] - 7s 119us/step - loss: 0.0314 - acc: 0.9909
Epoch 4/5
60000/60000 [==============================] - 7s 120us/step - loss: 0.0231 - acc: 0.9931
Epoch 5/5
10000/10000 [==============================] - 1s 137us/step
[ 6 ] {'train_accuracy': 0.9974666666666666, 'test_accuracy': 0.9867, 'feature_count': [32, 64, 128], 'conv_size': (3, 3), 'padding': 'same', 'activation': 'softsign', 'pool_size': (2, 2), 'loss': 'categorical_crossentropy', 'optimizer': <class 'keras.optimizers.Adamax'>}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_325 (Conv2D)          (None, 28, 28, 32)        320       
_________________________________________________________